In [ ]:
import csv
import pandas as pd
import re
import os
import numpy as np
import pandas as pd

In [ ]:


top=10
p='/Top10/'
d='/Top10/'


dirlist = [item for item in os.listdir(p) if os.path.isdir(os.path.join(p, item))]

lenth = len(dirlist)
for index in range(0, lenth):
  project = dirlist[index]
  print(index)
  print(project)

  directory1=d+project
  directory=p+project
  print(directory)
# read the CSV file into a Pandas DataFrame
  file_path=directory1+'/_10_test_Of_Commits.csv'
  if os.path.exists(file_path):
   df = pd.read_csv(directory1+'/_10_test_Of_Commits.csv', header=None)
   Co_changed_Files_in_commit=[]

# loop through the rows in the DataFrame
   for index, row in df.iterrows():
    List_Files=[]
    # loop through the columns in the row
    for column_index, column_value in enumerate(row):
        # do something with the column value
        #print(f'Row {index}, Column {column_index}: {column_value}')
        List_Files.append(column_value)
        
    
    #print(List_Files)
    new_list = [x for x in List_Files if str(x) != 'nan']
    Co_changed_Files_in_commit.append(new_list)
    #print(Co_changed_Files)  
    
  print(Co_changed_Files_in_commit) 


  directory_path = directory+'/IaC_Files_Antecedent'
       
# loop through all files in the directory
  for filename in os.listdir(directory_path):
            
            
          #Antedecent_file=[]
    # check if the file is a CSV file
            if filename.endswith('.csv'):
                CSV_File=os.path.join(directory_path, filename)

                with open(CSV_File, mode='r') as infile:

                             empty=''
                             reader = csv.reader(infile)
    # read the header row
                             header = next(reader)
    # add new column headers
                             header.extend(['Precision', 'Recall','MRR'])

                             rows = []
                             i=0
                             for row in reader:
                     
                               if i >= 1:
                                   row.extend([empty, empty,empty ])
                                   rows.append(row)
                     # add data to new columns from the lists
                               else:
                                   rows.append(row)
                     
                                   i+=1

    # create a list of the rows with the new columns
                             
     # open the CSV file for writing
                with open(CSV_File, mode='w', newline='') as outfile:
                               writer = csv.writer(outfile)
    # write the updated header row
                               writer.writerow(header)
                               writer.writerows(rows)

  target = '_(IaC)_'
  other = '_(Other)_'
  Only_IaC_files=[]
  Antedecent=[]
  Consequent_ranked=[]

  count=0
  for commit in Co_changed_Files_in_commit:
    
   Only_IaC_files=[]
   if len(commit)>1:
    
    count+=1
    print('this is commit ::',count)
    print(commit)
    print(len(commit))
    #Only_IaC_files=[]
    #print(commit)
    # take every iaC file found in commit
    for file in commit:
       # print(file)
        if target in file:
            #print(file)
            Only_IaC_files.append(file)
    

    for iac in Only_IaC_files:
        print("Looking for this file::",iac)
        
        files_in_commit=[]
        print(commit)
        
# loop through all files in the directory
        for filename in os.listdir(directory_path):
            
            
         
    # check if the file is a CSV file
            if filename.endswith('.csv'):
                CSV_File=os.path.join(directory_path, filename)

                with open(CSV_File, mode='r') as infile:
                    Antedecent=[]
                    Consequent_ranked=[]
                    print(CSV_File)
    # create a csv.DictReader object
                    reader = csv.DictReader(infile)
    # loop through the rows in the CSV file
                    for row in reader:
                      # get the values for the specified columns
                        PageRank_Files = row['PageRank_Files']
                        Consequent_ranked.append(PageRank_Files)
                        IaC_file = row['IaC_File']
                        Antedecent.append(IaC_file)
    
                    Antedecent_file_1 = set(Antedecent)
       


              ## check if iac equal to antecedent
                    Antedecent_file = list(set(Antedecent_file_1))
                    antec = str(Antedecent_file[0]) 
                    #print('antec',antec)
                    #print('iac',iac)
                    if  iac in antec: 

                       print('yes')
                 #print(iac)


                 ## compare Consequent_ranked with commit files
                 ## remove the iac from the commit
                       top5_in_80=Consequent_ranked[:top]
                       #top5_in_80=Consequent_ranked[:15]
                       print('top5_in_80',top5_in_80)
                       print('commit',commit)
                       files_in_commit=list(commit)
                       files_in_commit.remove(iac)
                       print('commit',commit)
                       print('files_in_commit',files_in_commit)

                 ## see if any recommeded files are found in the commit

                    
 
                       common_elements = set(files_in_commit) & set(top5_in_80)
                       common_elements_recall = set(files_in_commit) & set(top5_in_80)

                       print('common_elements',len(common_elements))
                       print('common_elements',len(common_elements_recall))





               
                       PRECISION= len(common_elements) / top
                       print('PRECISION=',PRECISION)

           
                       RECALL= len(common_elements_recall) / len(files_in_commit)
                       print('RECALL=',RECALL)

                       def compute_MRRold(recommended_files,ground_truth):
                            reciprocal_ranks = []
                            for i in range(len(recommended_files)):
                                ranks = [rank + 1 for rank, file in enumerate(recommended_files[i]) if file in ground_truth]
                                if len(ranks) > 0:
                                    reciprocal_ranks.append(1.0 / ranks[0])

                            mrr = np.mean(reciprocal_ranks)
                            return mrr
                       
                       def compute_MRR(recommended_ranked_lists, truths):
                            reciprocal_ranks = []

                            for truth, recommended_list in zip(truths, recommended_ranked_lists):
                                rank = next((i + 1 for i, item in enumerate(recommended_list) if item in truth),
                                            0)
                                if rank > 0:
                                    reciprocal_ranks.append(1.0 / rank)

                            if reciprocal_ranks:
                                mrr = sum(reciprocal_ranks) / len(reciprocal_ranks)
                            else:
                                mrr = 0

                            return mrr
                       
                       
   
                       MRR = compute_MRR([top5_in_80], [files_in_commit])
                       print("MRR", MRR)
                      
                      
                       with open(CSV_File, 'a', newline='') as csvfile:

    # create a writer object
                            csvwriter = csv.writer(csvfile)

    # specify the data to be appended
                            row = [PRECISION, RECALL, MRR]

    # write the row to the specific columns in the CSV file
                            csvwriter.writerow([None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None, row[0],row[1],row[2]])


                     

